In [4]:
import re
import collections

class BytePairEncoding:
    def __init__(self):
        self.bpe_codes = {}
        self.bpe_codes_reverse = {}
        self.vocabulary = set()

    def create_vocab(self, sentence):
        words = sentence.split()
        vocab = {}

        for word in words:
            characters = list(word)
            characters.append('$')
            vocab[' '.join(characters)] = vocab.get(' '.join(characters), 0) + 1

        return vocab

    def get_stats(self, vocab):
        pairs = collections.defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols)-1):
                pairs[symbols[i], symbols[i+1]] += freq
        return pairs

    def merge_vocab(self, pair, v_in):
        v_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in v_in:
            w_out = p.sub(''.join(pair), word)
            v_out[w_out] = v_in[word]
        return v_out

    def learn_vocabulary(self, corpus, num_merges):
        train_data = self.create_vocab(corpus)

        for i in range(num_merges):
            pairs = self.get_stats(train_data)
            best = max(pairs, key=pairs.get)
            train_data = self.merge_vocab(best, train_data)

            self.bpe_codes[best] = i
            self.bpe_codes_reverse[best[0] + best[1]] = best

        self.vocabulary = set(train_data.keys())

    def get_pairs(self, word):
        pairs = set()
        prev_char = word[0]
        for char in word[1:]:
            pairs.add((prev_char, char))
            prev_char = char
        return pairs

    def tokenize(self, sentence):
        words = sentence.split()
        tokenized_sentence = []

        for word in words:
            word_tokens = self._tokenize_word(word)
            tokenized_sentence.extend(word_tokens)

        return tokenized_sentence

    def _tokenize_word(self, orig):
        word = tuple(orig) + ('$',)

        pairs = self.get_pairs(word)

        if not pairs:
            return [orig]

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word) - 1 and word[i+1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            print("word after merging: {}".format(word))
            if len(word) == 1:
                break
            else:
                pairs = self.get_pairs(word)

        # if word[-1] == '$':
        #     word = word[:-1]
        # elif word[-1].endswith('$'):
        #     word = word[:-1] + (word[-1].replace('$', ''),)

        return list(word)

    def save_vocabulary(self, file_path="vocabulary.txt"):
        with open(file_path, 'w') as file:
            for token in self.vocabulary:
                file.write(token + '\n')

    def save_merge_rules(self, file_path="merge_rules.txt"):
        with open(file_path, 'w') as file:
            for rule, merge_id in self.bpe_codes.items():
                file.write(f"{rule[0]},{rule[1]}: {merge_id}\n")

    def tokenize_and_save(self, test_samples, file_path="tokenized_samples.txt"):
        with open(file_path, 'w') as file:
            for sample in test_samples:
                tokens = self.tokenize(sample)
                file.write('\n'.join(tokens) + '\n')

# Example usage:
bpe = BytePairEncoding()
file_path = '/content/corpus.txt'
corpus=""
try:
    with open(file_path, 'r') as file:
        for line in file:
            corpus = corpus +" "+ line.strip()
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")
num_merges = 100
bpe.learn_vocabulary(corpus, num_merges)

# Save Vocabulary
bpe.save_vocabulary()

# Save Merge Rules
bpe.save_merge_rules()

test_samples = ["test sample 1 lowest lowing"]
bpe.tokenize_and_save(test_samples)

word after merging: ('t', 'e', 's', 't$')
word after merging: ('t', 'es', 't$')
word after merging: ('s', 'a', 'm', 'p', 'l', 'e$')
word after merging: ('s', 'a', 'm', 'p', 'le$')
word after merging: ('l', 'o', 'w', 'e', 's', 't$')
word after merging: ('l', 'ow', 'e', 's', 't$')
word after merging: ('l', 'ow', 'es', 't$')
word after merging: ('l', 'o', 'w', 'in', 'g', '$')
word after merging: ('l', 'o', 'w', 'ing', '$')
word after merging: ('l', 'o', 'w', 'ing$')
word after merging: ('l', 'ow', 'ing$')
